# CICE Loaders

In [4]:
def open_by_run(run_name, vars):
    """ Concat any ds provided for this run into one ds, and add area and geo coordinates """

    import xarray as xr
    import cf_xarray as cfxr
    
    open_by_name = lambda name : catalog[name].search(
        variable=vars
    ).to_dask(
        xarray_open_kwargs={"chunks":{'time':'auto','ni':-1,'nj':-1},"decode_coords":False},
        xarray_combine_by_coords_kwargs={'compat':'override','data_vars':'minimal', 'coords':'minimal'}
    )

    #get the data for each run of this config 
    cice_ds=xr.concat([
        open_by_name(iName) for iName in RUNS[run_name]
    ], dim='time'
    )
    
    #We also want the area field, but its not time dependent.
    # get the path of the first file with the area field, the geolon field and the geolat field and open it
    area_ds=xr.merge([
        xr.open_dataset(
            catalog[RUNS[run_name][0]].search(variable=iVar).df.path[0]
        ).drop('time')
    for iVar in VARS_2D])

    #Label the lats and lons
    cice_ds.coords['ni'] = area_ds['xt_ocean'].values
    cice_ds.coords['nj'] = area_ds['yt_ocean'].values
    
    #Copy attributes for cf compliance
    cice_ds.ni.attrs=area_ds.xt_ocean.attrs
    cice_ds.nj.attrs=area_ds.yt_ocean.attrs
    
    cice_ds = cice_ds.rename(({'ni':'xt_ocean', 'nj':'yt_ocean'}))
    
    # Add the geolon, geolat, and area as extra co-ordinates fields from area_t
    
    cice_ds=cice_ds.assign_coords({
        'geolat_t':area_ds.geolat_t,
        'geolon_t':area_ds.geolon_t,
        'area_t':area_ds.area_t
    })

    # cice_ds['area_t']=area_ds['area_t']

    #cice timestamps are also misleading:
    cice_ds['time'] = cice_ds.time.to_pandas() - timedelta(days = 1)
       
    return cice_ds